In [1]:
import os

import pandas as pd
import polars as pl
import numpy as np
import seaborn as sns
import sgpp, sgml, sgutil

from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler, MinMaxScaler

In [2]:
expr_dic = {}
for i in ['sunshine', 'cloud', 'dewpoint', 'chp', 'cos_wd', 'sin_wd']:
    for j in range(1, 7):
        expr_dic['{}_{}'.format(i, j)] =  pl.col(i).shift(j).fill_null(strategy = 'backward')
for i in ['pressure', 'maxtemp', 'temparature', 'mintemp', 'dewpoint', 'humidity', 'cloud', 'sunshine', 'windspeed', 'cos_wd', 'sin_wd', 'chp']:
    expr_dic['{}_ma24'.format(i)] = pl.mean(i).rolling(index_column = 'id', period = '24i', closed = 'left').fill_null(strategy = 'backward')

In [3]:
p1 = make_pipeline(
    sgpp.PolarsProcessor(predefined_types = {'id': pl.Int64}),
    sgpp.ExprProcessor({
        'winddirection': pl.col('winddirection').fill_null(strategy = 'forward'),
        'windspeed': pl.col('windspeed').fill_null(strategy = 'forward')
    }),
    sgpp.ExprProcessor({
        'cos_wd': (pl.col('winddirection') / 180 * np.pi).cos() * pl.col('windspeed'),
        'sin_wd': (pl.col('winddirection') / 180 * np.pi).sin() * pl.col('windspeed'),
        'chp': pl.col('cloud') * pl.col('humidity') / pl.col('pressure'),
        'expected_day': (pl.col('id') % 365) + 1,
        'sin_ed': ((pl.col('id') % 365) / 365 * np.pi).sin(),
        'year': pl.col('id') // 365, 
    })
)
df_train = p1.fit_transform(['data/train.csv'])
df_test = p1.transform(['data/test.csv'])
p2 = make_pipeline(
    sgpp.ExprProcessor(expr_dic),
    sgpp.PandasCoverter(index_col = 'id')
)
df_all = p2.fit_transform(
    pl.concat([df_train, df_test], how = 'align')
)

p3 = make_pipeline(
    sgpp.ApplyWrapper(
        StandardScaler().set_output(transform='pandas'), 
        ['pressure', 'maxtemp', 'temparature', 'mintemp', 'dewpoint', 'humidity', 'cloud', 'sunshine', 'windspeed'] + ['cos_wd', 'sin_wd', 'chp'] + list(expr_dic.keys()),
    ),
    sgpp.ApplyWrapper(
        MinMaxScaler().set_output(transform='pandas'), ['year', 'expected_day', 'winddirection']
    )
)
df_all = p3.fit_transform(df_all)

df_train = df_all.loc[df_all['rainfall'].notna()]
df_test = df_all.loc[df_all['rainfall'].isna()].drop(columns = ['rainfall'])

In [4]:
df_org = make_pipeline(
    sgpp.PolarsProcessor(predefined_types = {'id': pl.Int64, 'day': pl.Int16, 'rainfall': pl.String}),
).fit_transform(['data/Rainfall.csv']).rename(
    lambda x: x.strip()
)
df_org = make_pipeline(
    sgpp.ExprProcessor({
        'winddirection': pl.col('winddirection').fill_null(strategy = 'forward'),
        'windspeed': pl.col('windspeed').fill_null(strategy = 'forward')
    }),
    sgpp.ExprProcessor({
        'id': pl.arange(1, pl.col('day').len() + 1),
        'sin_ed': (pl.arange(1, pl.col('day').len() + 1) / 365 * np.pi).sin(),
        'cos_wd': (pl.col('winddirection') / 180 * np.pi).cos() * pl.col('windspeed'),
        'sin_wd': (pl.col('winddirection') / 180 * np.pi).sin() * pl.col('windspeed'),
        'chp': pl.col('cloud') * pl.col('humidity') / pl.col('pressure'),
        'rainfall': pl.col('rainfall').replace({'yes': 1, 'no': 0}).cast(pl.Int8),
    }),
    sgpp.ExprProcessor(expr_dic),
    sgpp.PandasCoverter(), 
).fit_transform(df_org).assign(
    expected_day = lambda x: x.index + 1,
    year = -1
).pipe(
    lambda x: x.set_index(-(len(x) - x.index))
)
df_org = p3.transform(df_org)

In [5]:
target = 'rainfall'
sc = sgutil.SGCache('img', 'result')
_ = sc.cache_result(
    'target', lambda : df_train[target].sort_index()
)

In [6]:
len(df_test.columns)

65

In [7]:
import lightgbm as lgb
import xgboost as xgb
import catboost as cb
from sklearn.metrics import roc_auc_score
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import StratifiedKFold, StratifiedShuffleSplit

def get_validation_splitter(validation_fraction):
    return lambda x: train_test_split(x, test_size = validation_fraction)

def include_org(df, include_org = False):
    return pd.concat([df, df_org]) if include_org else df

config = {
    'predict_func': lambda m, df, X: pd.Series(m.predict_proba(df[X])[:, 1], index = df.index),
    'score_func': lambda df, prds: roc_auc_score(df[target], prds),
    'validation_splitter': get_validation_splitter,
    'progress_callback': sgml.ProgressCallBack(), 
    'return_train_scores': True,
    'train_data_proc': include_org,
    'y': target,
}

lr_adapter = sgml.SklearnAdapter(LogisticRegression)
svc_adapter = sgml.SklearnAdapter(SVC)
knn_adapter = sgml.SklearnAdapter(KNeighborsClassifier)
lgb_adapter = sgml.LGBMAdapter(lgb.LGBMClassifier)
xgb_adapter = sgml.XGBAdapter(xgb.XGBClassifier)
cb_adapter = sgml.CBAdapter(cb.CatBoostClassifier)

skf = StratifiedKFold(5, random_state = 123, shuffle = True)
ss = StratifiedShuffleSplit(1, random_state = 123)

In [8]:
from sklearn.feature_selection import SequentialFeatureSelector
X_all = [i for i in df_test.columns.tolist() if i not in ['day']]

# Forward Feature Selection wrapping Logistic Regression

In [9]:
sfs = sc.cache_result(
    'ff_sfs_lr',
    lambda : SequentialFeatureSelector(
        estimator = LogisticRegression(), direction = 'forward', scoring = 'roc_auc', cv = skf, n_jobs = -1
    ).fit(df_train[X_all], df_train[target])
)
np.array(X_all)[sfs.get_support()]

array(['sin_ed', 'pressure', 'maxtemp', 'temparature', 'mintemp',
       'dewpoint', 'sunshine', 'windspeed', 'chp', 'sunshine_1',
       'sunshine_2', 'sunshine_3', 'sunshine_4', 'cloud_3', 'cloud_4',
       'cloud_5', 'cloud_6', 'dewpoint_1', 'dewpoint_5', 'chp_1', 'chp_3',
       'chp_5', 'cos_wd_3', 'cos_wd_4', 'cos_wd_5', 'sin_wd_2',
       'maxtemp_ma24', 'temparature_ma24', 'dewpoint_ma24',
       'humidity_ma24', 'sunshine_ma24', 'year'], dtype='<U16')

In [10]:
hparams = {
    'model_params': {},
    'X_num': np.array(X_all)[sfs.get_support()].tolist()
}
result = sgml.cv(df_train, skf, hparams, config, lr_adapter, result_proc = [sgml.lr_learning_result])
np.mean(result['valid_scores']), np.std(result['valid_scores'])

Fold:   0%|          | 0/5 [00:00<?, ?it/s]

(0.8972053872053873, 0.012120648342876662)

In [11]:
df_coef = pd.concat([i['coef'] for i in result['model_result']], axis=1).agg(['mean', 'std'], axis = 1).assign(
    CV = lambda x: x['std'] / x['mean'].abs()
).sort_values('CV', ascending = False)
df_coef.iloc[:10]

,mean,std,CV
dewpoint_ma24,-0.020234,0.113573,5.612868
maxtemp_ma24,-0.024915,0.081302,3.263118
cos_wd_3,-0.014448,0.043408,3.004357
sunshine_3,-0.018969,0.033022,1.740821
sin_wd_2,-0.015010,0.022889,1.524927
dewpoint_1,0.052131,0.071972,1.380610
chp_1,-0.039389,0.049762,1.263360
maxtemp,0.127891,0.153483,1.200112
temparature_ma24,0.365757,0.205129,0.560833
dewpoint_5,-0.063698,0.031295,0.491304


In [12]:
sc.cv_result('lr_sfs', hparams, lr_adapter, result['valid_prd'])

In [13]:
hparams = {
    'model_params': {},
    'X_num': df_coef.iloc[8:].index.tolist()
}
result = sgml.cv(df_train, skf, hparams, config, lr_adapter, result_proc = [sgml.lr_learning_result])
np.mean(result['valid_scores']), np.std(result['valid_scores'])

Fold:   0%|          | 0/5 [00:00<?, ?it/s]

(0.8983557800224468, 0.011976255278421333)

In [14]:
df_coef = pd.concat([i['coef'] for i in result['model_result']], axis=1).agg(['mean', 'std'], axis = 1).assign(
    CV = lambda x: x['std'] / x['mean'].abs()
).sort_values('CV', ascending = False)
df_coef.iloc[:10]

,mean,std,CV
dewpoint_5,-0.060115,0.045998,0.765164
temparature,-0.148630,0.109471,0.736534
temparature_ma24,0.339612,0.173672,0.511386
cos_wd_5,0.098969,0.045272,0.457440
chp_3,-0.258010,0.096879,0.375485
cloud_6,0.065133,0.023313,0.357921
mintemp,-0.333428,0.118167,0.354401
cloud_5,0.185984,0.061321,0.329710
sunshine_2,0.115192,0.037786,0.328024
cloud_4,0.149698,0.045243,0.302226


In [15]:
sc.cv_result('lr_sfs2', hparams, lr_adapter, result['valid_prd'])

In [16]:
hparams = {
    'model_params': {},
    'X_num': df_coef.iloc[4:].index.tolist()
}
result = sgml.cv(df_train, skf, hparams, config, lr_adapter, result_proc = [sgml.lr_learning_result])
np.mean(result['valid_scores']), np.std(result['valid_scores'])

Fold:   0%|          | 0/5 [00:00<?, ?it/s]

(0.8985185185185184, 0.01312478134285176)

In [17]:
sc.cv_result('lr_sfs3', hparams, lr_adapter, result['valid_prd'])

In [18]:
df_coef = pd.concat([i['coef'] for i in result['model_result']], axis=1).agg(['mean', 'std'], axis = 1).assign(
    CV = lambda x: x['std'] / x['mean'].abs()
).sort_values('CV', ascending = False)
df_coef.iloc[:10]

,mean,std,CV
cloud_6,0.069157,0.025011,0.361659
sunshine_2,0.108589,0.038656,0.355989
chp_3,-0.283220,0.088740,0.313325
cloud_4,0.156865,0.045290,0.288716
pressure,-0.134968,0.037416,0.277223
sin_ed,-0.948355,0.226483,0.238817
sunshine_4,0.216750,0.051374,0.237018
cloud_5,0.232140,0.053793,0.231728
year,0.554469,0.127472,0.229900
chp_5,-0.200099,0.045552,0.227648


# Forward Feature Selection wrapping Logistic Regression Aumenting Org

In [19]:
sfs = sc.cache_result(
    'ff_sfs_lr2',
    lambda : pd.concat([df_train, df_org]).pipe(
        lambda x: SequentialFeatureSelector(
            estimator = LogisticRegression(), direction = 'forward', scoring = 'roc_auc', cv = skf, n_jobs = -1
        ).fit(x[X_all], x[target])
    )
)
np.array(X_all)[sfs.get_support()]

array(['sin_ed', 'pressure', 'maxtemp', 'temparature', 'mintemp',
       'dewpoint', 'humidity', 'cloud', 'sunshine', 'windspeed', 'cos_wd',
       'chp', 'sunshine_1', 'sunshine_2', 'sunshine_3', 'sunshine_4',
       'sunshine_5', 'cloud_2', 'cloud_3', 'cloud_4', 'dewpoint_2',
       'dewpoint_6', 'chp_2', 'chp_4', 'chp_6', 'cos_wd_3', 'sin_wd_2',
       'maxtemp_ma24', 'temparature_ma24', 'mintemp_ma24', 'year',
       'winddirection'], dtype='<U16')

In [20]:
hparams = {
    'model_params': {},
    'X_num': np.array(X_all)[sfs.get_support()].tolist(),
    'train_data_proc_param': {'include_org': True}
}
result = sgml.cv(df_train, skf, hparams, config, lr_adapter, result_proc = [sgml.lr_learning_result])
sc.cv_result('lr_sfs_a', hparams, lr_adapter, result['valid_prd'])
np.mean(result['valid_scores']), np.std(result['valid_scores'])

Fold:   0%|          | 0/5 [00:00<?, ?it/s]

(0.8947811447811448, 0.012302086080667738)

In [21]:
df_coef = pd.concat([i['coef'] for i in result['model_result']], axis=1).agg(['mean', 'std'], axis = 1).assign(
    CV = lambda x: x['std'] / x['mean'].abs()
).sort_values('CV', ascending = False)
df_coef.iloc[:10]

,mean,std,CV
maxtemp,-0.021097,0.149820,7.101481
cos_wd_3,-0.010459,0.045743,4.373663
mintemp_ma24,0.040715,0.160245,3.935732
sunshine_5,0.019164,0.033826,1.765063
chp_6,-0.022549,0.038773,1.719525
sin_wd_2,-0.022583,0.030652,1.357293
maxtemp_ma24,0.119041,0.137335,1.153678
chp_4,0.042263,0.034726,0.821670
chp,0.069188,0.052726,0.762075
temparature,-0.166954,0.126749,0.759184


In [22]:
hparams = {
    'model_params': {},
    'X_num': df_coef.iloc[6:].index.tolist(),
    'train_data_proc_param': {'include_org': True}
}
result = sgml.cv(df_train, skf, hparams, config, lr_adapter, result_proc = [sgml.lr_learning_result])
sc.cv_result('lr_sfs_a2', hparams, lr_adapter, result['valid_prd'])
np.mean(result['valid_scores']), np.std(result['valid_scores'])

Fold:   0%|          | 0/5 [00:00<?, ?it/s]

(0.8962906846240178, 0.01146601290644487)

In [23]:
df_coef = pd.concat([i['coef'] for i in result['model_result']], axis=1).agg(['mean', 'std'], axis = 1).assign(
    CV = lambda x: x['std'] / x['mean'].abs()
).sort_values('CV', ascending = False)
df_coef.iloc[:10]

,mean,std,CV
chp_4,0.034988,0.042251,1.207573
chp,0.054333,0.046949,0.864086
sunshine_3,0.048751,0.035849,0.735355
dewpoint_2,-0.100342,0.063914,0.636965
temparature,-0.184064,0.098829,0.536927
maxtemp_ma24,0.118299,0.062029,0.524339
chp_2,-0.134452,0.053239,0.395968
year,0.290991,0.111759,0.384062
cos_wd,-0.257909,0.063183,0.244983
dewpoint_6,0.340524,0.075403,0.221431


In [24]:
hparams = {
    'model_params': {},
    'X_num': df_coef.iloc[7:].index.tolist(),
    'train_data_proc_param': {'include_org': True}
}
result = sgml.cv(df_train, skf, hparams, config, lr_adapter, result_proc = [sgml.lr_learning_result])
sc.cv_result('lr_sfs_a3', hparams, lr_adapter, result['valid_prd'])
np.mean(result['valid_scores']), np.std(result['valid_scores'])

Fold:   0%|          | 0/5 [00:00<?, ?it/s]

(0.8971043771043771, 0.011150089212136121)

# Backward Feature Selection wrapping Logistic Regression

In [25]:
sfs = sc.cache_result(
    'bf_sfs_lr',
    lambda : SequentialFeatureSelector(
        estimator = LogisticRegression(), direction = 'backward', scoring = 'roc_auc', cv = skf, n_jobs = -1
    ).fit(df_train[X_all], df_train[target]), rerun = False
)
np.array(X_all)[sfs.get_support()]

array(['sin_ed', 'pressure', 'maxtemp', 'temparature', 'mintemp',
       'dewpoint', 'sunshine', 'windspeed', 'cos_wd', 'sin_wd', 'chp',
       'sunshine_1', 'sunshine_2', 'sunshine_4', 'cloud_3', 'cloud_4',
       'cloud_5', 'dewpoint_5', 'chp_3', 'chp_4', 'chp_5', 'cos_wd_3',
       'cos_wd_4', 'sin_wd_3', 'temparature_ma24', 'mintemp_ma24',
       'cloud_ma24', 'sunshine_ma24', 'cos_wd_ma24', 'sin_wd_ma24',
       'chp_ma24', 'year'], dtype='<U16')

In [26]:
hparams = {
    'model_params': {},
    'X_num': np.array(X_all)[sfs.get_support()].tolist(),
}
result = sgml.cv(df_train, skf, hparams, config, lr_adapter, result_proc = [sgml.lr_learning_result])
sc.cv_result('lr_bfs', hparams, lr_adapter, result['valid_prd'])
np.mean(result['valid_scores']), np.std(result['valid_scores'])

Fold:   0%|          | 0/5 [00:00<?, ?it/s]

(0.8971717171717172, 0.011681897853436847)

# Backward Feature Selection wrapping Logistic Regression Augumenting Org

In [27]:
sfs = sc.cache_result(
    'bf_sfs_lr2',
    lambda : pd.concat([df_train, df_org]).pipe(
        lambda x: SequentialFeatureSelector(
            estimator = LogisticRegression(max_iter = 500), direction = 'backward', scoring = 'roc_auc', cv = skf, n_jobs = -1
        ).fit(x[X_all], x[target])
    )
)
np.array(X_all)[sfs.get_support()]

array(['sin_ed', 'pressure', 'mintemp', 'dewpoint', 'humidity', 'cloud',
       'sunshine', 'windspeed', 'cos_wd', 'sin_wd', 'chp', 'sunshine_1',
       'sunshine_3', 'sunshine_4', 'sunshine_6', 'cloud_4', 'cloud_5',
       'dewpoint_2', 'dewpoint_6', 'chp_3', 'chp_5', 'chp_6', 'cos_wd_3',
       'sin_wd_1', 'sin_wd_3', 'sin_wd_5', 'temparature_ma24',
       'mintemp_ma24', 'cos_wd_ma24', 'sin_wd_ma24', 'year',
       'winddirection'], dtype='<U16')

In [28]:
hparams = {
    'model_params': {},
    'X_num': np.array(X_all)[sfs.get_support()].tolist(),
    'train_data_proc_param': {'include_org': True}
}
result = sgml.cv(df_train, skf, hparams, config, lr_adapter, result_proc = [sgml.lr_learning_result])
sc.cv_result('lr_bfs_a', hparams, lr_adapter, result['valid_prd'])
np.mean(result['valid_scores']), np.std(result['valid_scores'])

Fold:   0%|          | 0/5 [00:00<?, ?it/s]

(0.8929517396184063, 0.009206929636066645)

# Forward Feature Selection wrapping LGBM

In [93]:
hparams = {
    'model_params': {'num_leaves': 7, 'n_estimators': 100, 'colsample_bytree': 0.75, 'learning_rate': 0.03},
    'X_num': X_all
}
result = sgml.cv(df_train, skf, hparams, config, lgb_adapter)
np.mean(result['valid_scores']), np.mean(result['train_scores'])

Fold:   0%|          | 0/5 [00:00<?, ?it/s]

(0.8888355780022446, 0.9411340838945005)

In [94]:
sfs = sc.cache_result(
    'ff_sfs_lgb',
    lambda : SequentialFeatureSelector(
        estimator = lgb.LGBMClassifier(verbose = 0, **hparams['model_params']), direction = 'forward', scoring = 'roc_auc', cv = skf
    ).fit(df_train[X_all], df_train[target])
)
np.array(X_all)[sfs.get_support()]

array(['sin_ed', 'maxtemp', 'temparature', 'mintemp', 'dewpoint',
       'sunshine', 'windspeed', 'sin_wd', 'chp', 'cloud_1', 'cloud_2',
       'cloud_3', 'cloud_5', 'cloud_6', 'dewpoint_1', 'dewpoint_2',
       'dewpoint_4', 'chp_3', 'chp_5', 'cos_wd_3', 'cos_wd_4', 'cos_wd_5',
       'sin_wd_2', 'sin_wd_6', 'maxtemp_ma24', 'temparature_ma24',
       'mintemp_ma24', 'dewpoint_ma24', 'humidity_ma24', 'sunshine_ma24',
       'year', 'expected_day'], dtype='<U16')

In [95]:
hparams = {
    'model_params': {'num_leaves': 7, 'n_estimators': 100, 'colsample_bytree': 0.75, 'learning_rate': 0.03},
    'X_num': np.array(X_all)[sfs.get_support()].tolist()
}
result = sgml.cv(df_train, skf, hparams, config, lgb_adapter)
sc.cv_result('lgb_sfs', hparams, lgb_adapter, result['valid_prd'])
np.mean(result['valid_scores']), np.mean(result['train_scores'])

Fold:   0%|          | 0/5 [00:00<?, ?it/s]

(0.8977104377104377, 0.9356549873737373)

In [96]:
df_imp = pd.concat(
    [i['feature_importance'] for i in result['model_result']], axis=1
).agg(['mean', 'std'], axis=1).assign(
    CV = lambda x: x['std'] / x['mean'].abs()
).sort_values('CV', ascending = False)
df_imp.iloc[:10]

,mean,std,CV
cloud_5,5.8,6.180615,1.065623
temparature,3.2,3.114482,0.973276
cos_wd_3,8.4,7.893035,0.939647
sin_wd,11.2,8.105554,0.723710
temparature_ma24,9.0,6.324555,0.702728
cloud_2,2.2,1.483240,0.674200
cos_wd_5,7.6,4.722288,0.621354
cloud_6,6.2,3.701351,0.596992
expected_day,9.4,5.224940,0.555845
mintemp,7.8,4.324350,0.554404


In [33]:
hparams = {
    'model_params': {'num_leaves': 7, 'n_estimators': 100, 'colsample_bytree': 0.75, 'learning_rate': 0.03},
    'X_num': df_imp.iloc[3:].index.tolist()
}
result = sgml.cv(df_train, skf, hparams, config, lgb_adapter)
sc.cv_result('lgb_sfs2', hparams, lgb_adapter, result['valid_prd'])
np.mean(result['valid_scores']), np.mean(result['train_scores'])

Fold:   0%|          | 0/5 [00:00<?, ?it/s]

(0.8973120089786757, 0.9356037808641975)

In [34]:
df_imp = pd.concat(
    [i['feature_importance'] for i in result['model_result']], axis=1
).agg(['mean', 'std'], axis=1).assign(
    CV = lambda x: x['std'] / x['mean'].abs()
).sort_values('CV', ascending = False)
df_imp.iloc[:10]

,mean,std,CV
cos_wd_5,9.6,8.173127,0.851367
dewpoint_1,6.0,4.062019,0.677003
mintemp_ma24,6.2,3.768289,0.607789
dewpoint_ma24,14.4,8.502941,0.590482
sin_ed,5.6,3.286335,0.586846
sin_wd_2,5.0,2.738613,0.547723
chp_5,9.6,5.176872,0.539257
dewpoint_4,12.0,5.916080,0.493007
sin_wd,13.6,6.693280,0.492153
temparature_ma24,7.6,3.646917,0.479857


# Forward Feature Selection wrapping LGBM Augumenting Org

In [35]:
hparams = {
    'model_params': {'num_leaves': 7, 'n_estimators': 100, 'colsample_bytree': 0.75, 'learning_rate': 0.03},
    'X_num': X_all
}
sfs = sc.cache_result(
    'ff_sfs_lgb2',
    lambda : pd.concat([df_train, df_org]).pipe(
        lambda x: SequentialFeatureSelector(
            estimator = lgb.LGBMClassifier(verbose = 0, **hparams['model_params']), direction = 'forward', scoring = 'roc_auc', cv = skf
        ).fit(x[X_all], x[target])
    )
)
np.array(X_all)[sfs.get_support()]

array(['sin_ed', 'pressure', 'maxtemp', 'temparature', 'mintemp',
       'dewpoint', 'humidity', 'cloud', 'sunshine', 'windspeed', 'chp',
       'sunshine_1', 'sunshine_5', 'cloud_2', 'cloud_3', 'cloud_6',
       'dewpoint_1', 'dewpoint_2', 'chp_2', 'chp_3', 'chp_4', 'chp_6',
       'cos_wd_1', 'sin_wd_5', 'maxtemp_ma24', 'temparature_ma24',
       'dewpoint_ma24', 'humidity_ma24', 'cloud_ma24', 'windspeed_ma24',
       'chp_ma24', 'year'], dtype='<U16')

In [36]:
hparams = {
    'model_params': {'num_leaves': 7, 'n_estimators': 100, 'colsample_bytree': 0.75, 'learning_rate': 0.03},
    'X_num': np.array(X_all)[sfs.get_support()].tolist()
}
result = sgml.cv(df_train, skf, hparams, config, lgb_adapter)
sc.cv_result('lgb_sfs_a', hparams, lgb_adapter, result['valid_prd'])
np.mean(result['valid_scores']), np.mean(result['train_scores'])

Fold:   0%|          | 0/5 [00:00<?, ?it/s]

(0.8927384960718294, 0.9361842382154884)

In [37]:
df_imp = pd.concat(
    [i['feature_importance'] for i in result['model_result']], axis=1
).agg(['mean', 'std'], axis=1).assign(
    CV = lambda x: x['std'] / x['mean'].abs()
).sort_values('CV', ascending = False)
df_imp.iloc[:10]

,mean,std,CV
chp_2,5.0,6.442049,1.288410
year,4.0,4.183300,1.045825
sin_ed,4.4,4.560702,1.036523
cloud_2,2.2,2.167948,0.985431
temparature,3.2,2.949576,0.921743
chp_4,3.8,2.489980,0.655258
humidity,12.2,7.949843,0.651626
sunshine_5,7.8,4.549725,0.583298
cloud_6,6.6,3.781534,0.572960
sin_wd_5,6.2,3.346640,0.539781


In [38]:
hparams = {
    'model_params': {'num_leaves': 7, 'n_estimators': 100, 'colsample_bytree': 0.75, 'learning_rate': 0.03},
    'X_num': df_imp.iloc[6:].index.tolist()
}
result = sgml.cv(df_train, skf, hparams, config, lgb_adapter)
sc.cv_result('lgb_sfs_a2', hparams, lgb_adapter, result['valid_prd'])
np.mean(result['valid_scores']), np.mean(result['train_scores'])

Fold:   0%|          | 0/5 [00:00<?, ?it/s]

(0.891091470258137, 0.9354862864758697)

# Backward Feature Selection wrapping LGBM

In [39]:
hparams = {
    'model_params': {'num_leaves': 7, 'n_estimators': 100, 'colsample_bytree': 0.75, 'learning_rate': 0.03},
    'X_num': X_all
}
sfs = sc.cache_result(
    'bf_sfs_lgb',
    lambda : SequentialFeatureSelector(
        estimator = lgb.LGBMClassifier(verbose = 0, **hparams['model_params']), direction = 'backward', scoring = 'roc_auc', cv = skf
    ).fit(df_train[X_all], df_train[target])
)
np.array(X_all)[sfs.get_support()]

array(['temparature', 'mintemp', 'sunshine', 'windspeed', 'cos_wd', 'chp',
       'sunshine_1', 'sunshine_6', 'cloud_1', 'cloud_5', 'cloud_6',
       'dewpoint_3', 'chp_1', 'chp_2', 'chp_3', 'chp_5', 'cos_wd_3',
       'cos_wd_4', 'sin_wd_3', 'sin_wd_4', 'sin_wd_6', 'pressure_ma24',
       'maxtemp_ma24', 'temparature_ma24', 'dewpoint_ma24',
       'humidity_ma24', 'cloud_ma24', 'sunshine_ma24', 'sin_wd_ma24',
       'chp_ma24', 'year', 'winddirection'], dtype='<U16')

In [40]:
hparams = {
    'model_params': {'num_leaves': 7, 'n_estimators': 100, 'colsample_bytree': 0.75, 'learning_rate': 0.03},
    'X_num': np.array(X_all)[sfs.get_support()].tolist()
}
result = sgml.cv(df_train, skf, hparams, config, lgb_adapter)
sc.cv_result('lgb_bfs', hparams, lgb_adapter, result['valid_prd'])
np.mean(result['valid_scores']), np.mean(result['train_scores'])

Fold:   0%|          | 0/5 [00:00<?, ?it/s]

(0.8948512906846242, 0.9358101851851851)

# Forward Feature Selection wrapping XGB

In [50]:
hparams = {
    'model_params': {'max_depth': 3, 'n_estimators': 100, 'colsample_bytree': 0.75, 'learning_rate': 0.04},
    'X_num': X_all
}
result = sgml.cv(df_train, skf, hparams, config, xgb_adapter)
np.mean(result['valid_scores']), np.mean(result['train_scores'])

Fold:   0%|          | 0/5 [00:00<?, ?it/s]

(0.8905050505050506, 0.9449298540965207)

In [53]:
sfs = sc.cache_result(
    'ff_sfs_xgb',
    lambda : SequentialFeatureSelector(
        estimator = xgb.XGBClassifier(**hparams['model_params']), direction = 'forward', scoring = 'roc_auc', cv = skf
    ).fit(df_train[X_all], df_train[target])
)
np.array(X_all)[sfs.get_support()]

array(['sin_ed', 'maxtemp', 'temparature', 'mintemp', 'dewpoint',
       'sunshine', 'windspeed', 'chp', 'sunshine_3', 'sunshine_6',
       'cloud_1', 'cloud_6', 'dewpoint_2', 'dewpoint_5', 'chp_1', 'chp_3',
       'chp_4', 'chp_6', 'cos_wd_1', 'cos_wd_4', 'cos_wd_5', 'sin_wd_2',
       'sin_wd_5', 'sin_wd_6', 'maxtemp_ma24', 'temparature_ma24',
       'dewpoint_ma24', 'humidity_ma24', 'sunshine_ma24',
       'windspeed_ma24', 'sin_wd_ma24', 'year'], dtype='<U16')

In [54]:
hparams = {
    'model_params': {'max_depth': 3, 'n_estimators': 100, 'colsample_bytree': 0.75, 'learning_rate': 0.04},
    'X_num': np.array(X_all)[sfs.get_support()].tolist()
}
result = sgml.cv(df_train, skf, hparams, config, xgb_adapter)
sc.cv_result('xgb_sfs', hparams, xgb_adapter, result['valid_prd'])
np.mean(result['valid_scores']), np.mean(result['train_scores'])

Fold:   0%|          | 0/5 [00:00<?, ?it/s]

(0.8978843995510661, 0.9390263748597082)

In [55]:
df_imp = pd.concat(
    [i['feature_importance'] for i in result['model_result']], axis=1
).agg(['mean', 'std'], axis=1).assign(
    CV = lambda x: x['std'] / x['mean'].abs()
).sort_values('CV', ascending = False)
df_imp.iloc[:10]

,mean,std,CV
cloud_6,0.011306,0.006427,0.568440
temparature,0.052128,0.027954,0.536251
maxtemp,0.052110,0.024613,0.472330
mintemp,0.039521,0.017824,0.451009
sin_wd_ma24,0.014777,0.006521,0.441330
chp_4,0.017471,0.007005,0.400959
chp_1,0.023313,0.008942,0.383574
sunshine_3,0.018504,0.005850,0.316163
cos_wd_5,0.019782,0.006072,0.306921
sin_ed,0.021846,0.006131,0.280636


In [61]:
hparams = {
    'model_params': {'max_depth': 3, 'n_estimators': 100, 'colsample_bytree': 0.75, 'learning_rate': 0.04},
    'X_num': df_imp.iloc[2:].index.tolist()
}
result = sgml.cv(df_train, skf, hparams, config, xgb_adapter)
sc.cv_result('xgb_sfs2', hparams, xgb_adapter, result['valid_prd'])
np.mean(result['valid_scores']), np.mean(result['train_scores'])

Fold:   0%|          | 0/5 [00:00<?, ?it/s]

(0.8981257014590348, 0.9392839856902355)

# Forward Feature Selection wrapping LGBM Augumenting Org

In [63]:
hparams = {
    'model_params': {'max_depth': 3, 'n_estimators': 100, 'colsample_bytree': 0.75, 'learning_rate': 0.04},
    'X_num': X_all
}
sfs = sc.cache_result(
    'ff_sfs_xgb2',
    lambda : pd.concat([df_train, df_org]).pipe(
        lambda x: SequentialFeatureSelector(
            estimator = xgb.XGBClassifier(**hparams['model_params']), direction = 'forward', scoring = 'roc_auc', cv = skf
        ).fit(x[X_all], x[target])
    )
)
np.array(X_all)[sfs.get_support()]

array(['sin_ed', 'pressure', 'maxtemp', 'temparature', 'mintemp',
       'dewpoint', 'humidity', 'cloud', 'sunshine', 'windspeed', 'cos_wd',
       'sin_wd', 'chp', 'sunshine_1', 'sunshine_3', 'sunshine_6',
       'cloud_3', 'cloud_4', 'cloud_5', 'dewpoint_3', 'chp_2', 'chp_3',
       'cos_wd_1', 'cos_wd_2', 'cos_wd_6', 'sin_wd_2', 'temparature_ma24',
       'mintemp_ma24', 'cloud_ma24', 'sunshine_ma24', 'windspeed_ma24',
       'year'], dtype='<U16')

In [64]:
hparams = {
    'model_params': {'max_depth': 3, 'n_estimators': 100, 'colsample_bytree': 0.75, 'learning_rate': 0.04},
    'X_num': np.array(X_all)[sfs.get_support()].tolist(),
    'train_data_proc_param': {'include_org': True}
}
result = sgml.cv(df_train, skf, hparams, config, xgb_adapter)
sc.cv_result('xgb_sfs_a', hparams, xgb_adapter, result['valid_prd'])
np.mean(result['valid_scores']), np.mean(result['train_scores'])

Fold:   0%|          | 0/5 [00:00<?, ?it/s]

(0.8944612794612794, 0.9324633350398953)

In [65]:
df_imp = pd.concat(
    [i['feature_importance'] for i in result['model_result']], axis=1
).agg(['mean', 'std'], axis=1).assign(
    CV = lambda x: x['std'] / x['mean'].abs()
).sort_values('CV', ascending = False)
df_imp.iloc[:10]

,mean,std,CV
year,0.005335,0.004878,0.914231
cloud_4,0.017346,0.009222,0.531667
dewpoint_3,0.019117,0.010074,0.526965
temparature,0.036800,0.017721,0.481544
cos_wd_6,0.019855,0.008569,0.431595
sin_wd_2,0.014772,0.006127,0.414756
maxtemp,0.037074,0.011941,0.322093
cos_wd_2,0.020795,0.006611,0.317930
cos_wd_1,0.014534,0.004450,0.306173
mintemp,0.043675,0.012986,0.297342


In [71]:
hparams = {
    'model_params': {'max_depth': 3, 'n_estimators': 100, 'colsample_bytree': 0.75, 'learning_rate': 0.04},
    'X_num': df_imp.iloc[6:].index.tolist(),
    'train_data_proc_param': {'include_org': True}
}
result = sgml.cv(df_train, skf, hparams, config, xgb_adapter)
np.mean(result['valid_scores']), np.mean(result['train_scores'])

Fold:   0%|          | 0/5 [00:00<?, ?it/s]

(0.8930976430976431, 0.9315062672615255)

# Backward Feature Selection wrapping XGB

In [73]:
sfs = sc.cache_result(
    'bf_sfs_xgb',
    lambda : SequentialFeatureSelector(
        estimator = xgb.XGBClassifier(**hparams['model_params']), direction = 'backward', scoring = 'roc_auc', cv = skf
    ).fit(df_train[X_all], df_train[target])
)
np.array(X_all)[sfs.get_support()]

array(['pressure', 'dewpoint', 'sunshine', 'windspeed', 'cos_wd', 'chp',
       'sunshine_1', 'sunshine_2', 'sunshine_3', 'cloud_1', 'cloud_3',
       'cloud_4', 'cloud_5', 'cloud_6', 'dewpoint_2', 'dewpoint_3',
       'chp_3', 'chp_4', 'chp_6', 'cos_wd_1', 'cos_wd_2', 'cos_wd_4',
       'sin_wd_2', 'sin_wd_6', 'pressure_ma24', 'maxtemp_ma24',
       'temparature_ma24', 'mintemp_ma24', 'humidity_ma24',
       'sunshine_ma24', 'windspeed_ma24', 'chp_ma24'], dtype='<U16')

In [74]:
hparams = {
    'model_params': {'max_depth': 3, 'n_estimators': 100, 'colsample_bytree': 0.75, 'learning_rate': 0.04},
    'X_num': np.array(X_all)[sfs.get_support()].tolist()
}
result = sgml.cv(df_train, skf, hparams, config, xgb_adapter)
sc.cv_result('xgb_bfs', hparams, xgb_adapter, result['valid_prd'])
np.mean(result['valid_scores']), np.mean(result['train_scores'])

Fold:   0%|          | 0/5 [00:00<?, ?it/s]

(0.8954040404040404, 0.9390477693602695)

# Stepwise Feature Selections wrapping Logistic Regression

In [102]:
from mlxtend import feature_selection as mfs

In [109]:
sfs = sc.cache_result(
    'ff_sfsf_lr',
    lambda : mfs.SequentialFeatureSelector(
        estimator = LogisticRegression(), k_features = 'best', forward = True, floating = True, scoring = 'roc_auc', cv = skf, n_jobs = -1
    ).fit(df_train[X_all], df_train[target])
)
np.array(sfs.k_feature_names_)

array(['sin_ed', 'pressure', 'mintemp', 'dewpoint', 'sunshine',
       'windspeed', 'chp', 'sunshine_1', 'sunshine_2', 'sunshine_4',
       'cloud_3', 'cloud_4', 'cloud_5', 'dewpoint_5', 'chp_5', 'cos_wd_4',
       'maxtemp_ma24', 'temparature_ma24', 'humidity_ma24', 'cos_wd_ma24',
       'sin_wd_ma24', 'year'], dtype='<U16')

In [111]:
hparams = {
    'model_params': {},
    'X_num': list(sfs.k_feature_names_)
}
result = sgml.cv(df_train, skf, hparams, config, lr_adapter, result_proc = [sgml.lr_learning_result])
sc.cv_result('lr_sfs_f', hparams, lr_adapter, result['valid_prd'])
np.mean(result['valid_scores']), np.std(result['valid_scores'])

Fold:   0%|          | 0/5 [00:00<?, ?it/s]

(0.8990460157126824, 0.01061797500616197)

In [113]:
sfs = sc.cache_result(
    'ff_sfsf_lr2',
    lambda : pd.concat([df_train, df_org]).pipe(
        lambda x: mfs.SequentialFeatureSelector(
            estimator = LogisticRegression(), k_features = 'best', forward = True, floating = True, scoring = 'roc_auc', cv = skf, n_jobs = -1
        ).fit(x[X_all], x[target])
    )
)
np.array(sfs.k_feature_names_)

array(['sin_ed', 'pressure', 'mintemp', 'dewpoint', 'humidity', 'cloud',
       'sunshine', 'windspeed', 'cos_wd', 'sunshine_1', 'cloud_3',
       'dewpoint_2', 'dewpoint_6', 'maxtemp_ma24', 'temparature_ma24',
       'mintemp_ma24', 'winddirection'], dtype='<U16')

In [114]:
hparams = {
    'model_params': {},
    'X_num': list(sfs.k_feature_names_)
}
result = sgml.cv(df_train, skf, hparams, config, lr_adapter, result_proc = [sgml.lr_learning_result])
sc.cv_result('lr_sfs_f2', hparams, lr_adapter, result['valid_prd'])
np.mean(result['valid_scores']), np.std(result['valid_scores'])

Fold:   0%|          | 0/5 [00:00<?, ?it/s]

(0.8960437710437711, 0.011426817196481703)

In [115]:
sfs = sc.cache_result(
    'bf_sfsf_lr',
    lambda : mfs.SequentialFeatureSelector(
        estimator = LogisticRegression(), k_features = 'best', forward = False, floating = True, scoring = 'roc_auc', cv = skf, n_jobs = -1
    ).fit(df_train[X_all], df_train[target])
)
np.array(sfs.k_feature_names_)

array(['sin_ed', 'mintemp', 'dewpoint', 'cloud', 'sunshine', 'windspeed',
       'chp', 'sunshine_1', 'sunshine_4', 'cloud_3', 'cloud_4',
       'cos_wd_4', 'temparature_ma24', 'humidity_ma24', 'cos_wd_ma24',
       'sin_wd_ma24', 'year'], dtype='<U16')

In [116]:
hparams = {
    'model_params': {},
    'X_num': list(sfs.k_feature_names_)
}
result = sgml.cv(df_train, skf, hparams, config, lr_adapter, result_proc = [sgml.lr_learning_result])
sc.cv_result('lr_bfs_f', hparams, lr_adapter, result['valid_prd'])
np.mean(result['valid_scores']), np.std(result['valid_scores'])

Fold:   0%|          | 0/5 [00:00<?, ?it/s]

(0.8991919191919191, 0.010802219887223165)

In [119]:
hparams = {
    'model_params': {'num_leaves': 7, 'n_estimators': 100, 'colsample_bytree': 0.75, 'learning_rate': 0.03},
    'X_num': X_all
}
sfs = sc.cache_result(
    'ff_sfs_lgb_f',
    lambda : mfs.SequentialFeatureSelector(
        estimator = lgb.LGBMClassifier(verbose = 0, **hparams['model_params']), k_features = 'best', forward = True, floating = True, scoring = 'roc_auc', cv = skf
    ).fit(df_train[X_all], df_train[target])

)
np.array(sfs.k_feature_names_)

array(['sin_ed', 'mintemp', 'sunshine', 'sin_wd', 'chp', 'cloud_1',
       'cloud_5', 'cloud_6', 'chp_3', 'sin_wd_6', 'maxtemp_ma24',
       'temparature_ma24', 'sunshine_ma24', 'chp_ma24'], dtype='<U16')

In [120]:
hparams = {
    'model_params': {'num_leaves': 7, 'n_estimators': 100, 'colsample_bytree': 0.75, 'learning_rate': 0.03},
    'X_num': list(sfs.k_feature_names_)
}
result = sgml.cv(df_train, skf, hparams, config, lgb_adapter)
sc.cv_result('lgb_sfs_f', hparams, lgb_adapter, result['valid_prd'])
np.mean(result['valid_scores']), np.mean(result['train_scores'])

Fold:   0%|          | 0/5 [00:00<?, ?it/s]

(0.8980948372615039, 0.9291661405723906)

In [121]:
hparams = {
    'model_params': {'num_leaves': 7, 'n_estimators': 100, 'colsample_bytree': 0.75, 'learning_rate': 0.03},
    'X_num': X_all
}
sfs = sc.cache_result(
    'bf_sfs_lgb_f',
    lambda : mfs.SequentialFeatureSelector(
        estimator = lgb.LGBMClassifier(verbose = 0, **hparams['model_params']), k_features = 'best', forward = False, floating = True, scoring = 'roc_auc', cv = skf
    ).fit(df_train[X_all], df_train[target])

)
np.array(sfs.k_feature_names_)

array(['sin_ed', 'temparature', 'mintemp', 'dewpoint', 'sunshine',
       'windspeed', 'chp', 'cloud_3', 'chp_3', 'chp_4', 'cos_wd_4',
       'maxtemp_ma24', 'temparature_ma24', 'humidity_ma24', 'cos_wd_ma24'],
      dtype='<U16')

In [122]:
hparams = {
    'model_params': {'num_leaves': 7, 'n_estimators': 100, 'colsample_bytree': 0.75, 'learning_rate': 0.03},
    'X_num': list(sfs.k_feature_names_)
}
result = sgml.cv(df_train, skf, hparams, config, lgb_adapter)
sc.cv_result('lgb_bfs_f', hparams, lgb_adapter, result['valid_prd'])
np.mean(result['valid_scores']), np.mean(result['train_scores'])

Fold:   0%|          | 0/5 [00:00<?, ?it/s]

(0.897135241301908, 0.9324726430976431)

In [123]:
hparams = {
    'model_params': {'max_depth': 3, 'n_estimators': 100, 'colsample_bytree': 0.75, 'learning_rate': 0.04},
    'X_num': X_all
}
sfs = sc.cache_result(
    'ff_sfs_xgb_f',
    lambda : mfs.SequentialFeatureSelector(
        estimator = xgb.XGBClassifier(**hparams['model_params']), k_features = 'best', forward = True, floating = True, scoring = 'roc_auc', cv = skf
    ).fit(df_train[X_all], df_train[target])

)
np.array(sfs.k_feature_names_)

array(['sin_ed', 'maxtemp', 'temparature', 'mintemp', 'dewpoint',
       'sunshine', 'windspeed', 'chp', 'sunshine_3', 'cloud_1', 'cloud_6',
       'dewpoint_2', 'cos_wd_4', 'cos_wd_5', 'sin_wd_2', 'sin_wd_6',
       'maxtemp_ma24', 'temparature_ma24', 'dewpoint_ma24',
       'humidity_ma24'], dtype='<U16')

In [124]:
hparams = {
    'model_params': {'max_depth': 3, 'n_estimators': 100, 'colsample_bytree': 0.75, 'learning_rate': 0.04},
    'X_num': list(sfs.k_feature_names_)
}
result = sgml.cv(df_train, skf, hparams, config, xgb_adapter)
sc.cv_result('xgb_sfs_f', hparams, xgb_adapter, result['valid_prd'])
np.mean(result['valid_scores']), np.mean(result['train_scores'])

Fold:   0%|          | 0/5 [00:00<?, ?it/s]

(0.8991835016835017, 0.9356292087542087)

In [125]:
hparams = {
    'model_params': {'max_depth': 3, 'n_estimators': 100, 'colsample_bytree': 0.75, 'learning_rate': 0.04},
    'X_num': X_all
}
sfs = sc.cache_result(
    'bf_sfs_xgb_f',
    lambda : mfs.SequentialFeatureSelector(
        estimator = xgb.XGBClassifier(**hparams['model_params']), k_features = 'best', forward = False, floating = True, scoring = 'roc_auc', cv = skf
    ).fit(df_train[X_all], df_train[target])

)
np.array(sfs.k_feature_names_)

array(['sin_ed', 'mintemp', 'dewpoint', 'sunshine', 'windspeed', 'chp',
       'sunshine_3', 'cloud_1', 'dewpoint_2', 'cos_wd_4', 'sin_wd_6',
       'dewpoint_ma24', 'humidity_ma24', 'sin_wd_ma24', 'year'],
      dtype='<U13')

In [126]:
hparams = {
    'model_params': {'max_depth': 3, 'n_estimators': 100, 'colsample_bytree': 0.75, 'learning_rate': 0.04},
    'X_num': list(sfs.k_feature_names_)
}
result = sgml.cv(df_train, skf, hparams, config, xgb_adapter)
sc.cv_result('xgb_bfs_f', hparams, xgb_adapter, result['valid_prd'])
np.mean(result['valid_scores']), np.mean(result['train_scores'])

Fold:   0%|          | 0/5 [00:00<?, ?it/s]

(0.8994893378226712, 0.9344342732884401)

In [147]:
hparams = {
    'model_params': {'C': 0.01, 'probability': True, 'kernel': 'linear'},
    'X_num': X_all
}
result = sgml.cv(df_train, skf, hparams, config, svc_adapter)
np.mean(result['valid_scores']), np.mean(result['train_scores'])

Fold:   0%|          | 0/5 [00:00<?, ?it/s]

(0.8915207631874298, 0.9011109357463525)

In [148]:
sfs = sc.cache_result(
    'ff_sfsf_lsvc',
    lambda : mfs.SequentialFeatureSelector(
        estimator = SVC(**hparams['model_params']), k_features = 'best', forward = True, floating = True, scoring = 'roc_auc', cv = skf, n_jobs = -1
    ).fit(df_train[X_all], df_train[target])
)
np.array(sfs.k_feature_names_)

array(['sin_ed', 'pressure', 'temparature', 'mintemp', 'dewpoint',
       'sunshine', 'windspeed', 'chp', 'sunshine_1', 'sunshine_2',
       'cloud_3', 'dewpoint_2', 'dewpoint_5', 'dewpoint_6', 'chp_5',
       'cos_wd_5', 'sin_wd_2', 'sin_wd_4', 'maxtemp_ma24',
       'temparature_ma24', 'dewpoint_ma24', 'humidity_ma24',
       'cos_wd_ma24', 'year'], dtype='<U16')

In [150]:
hparams = {
    'model_params': {'C': 0.01, 'probability': True, 'kernel': 'linear'},
    'X_num': list(sfs.k_feature_names_)
}
result = sgml.cv(df_train, skf, hparams, config, svc_adapter)
sc.cv_result('lsvc_sfs_f', hparams, svc_adapter, result['valid_prd'])
np.mean(result['valid_scores']), np.mean(result['train_scores'])

Fold:   0%|          | 0/5 [00:00<?, ?it/s]

(0.8986139169472503, 0.8995186237373737)

In [151]:
sfs = sc.cache_result(
    'bf_sfsf_lsvc',
    lambda : mfs.SequentialFeatureSelector(
        estimator = SVC(**hparams['model_params']), k_features = 'best', forward = False, floating = True, scoring = 'roc_auc', cv = skf, n_jobs = -1
    ).fit(df_train[X_all], df_train[target])
)
np.array(sfs.k_feature_names_)

array(['sin_ed', 'pressure', 'temparature', 'mintemp', 'dewpoint',
       'humidity', 'cloud', 'sunshine', 'windspeed', 'cos_wd',
       'sunshine_1', 'cloud_3', 'cos_wd_4', 'cos_wd_5', 'sin_wd_4',
       'temparature_ma24', 'mintemp_ma24', 'humidity_ma24', 'cos_wd_ma24',
       'year'], dtype='<U16')

In [152]:
hparams = {
    'model_params': {'C': 0.01, 'probability': True, 'kernel': 'linear'},
    'X_num': list(sfs.k_feature_names_)
}
result = sgml.cv(df_train, skf, hparams, config, svc_adapter)
sc.cv_result('lsvc_bfs_f', hparams, svc_adapter, result['valid_prd'])
np.mean(result['valid_scores']), np.mean(result['train_scores'])

Fold:   0%|          | 0/5 [00:00<?, ?it/s]

(0.899074074074074, 0.8999372194163862)

In [172]:
hparams = {
    'model_params': {'C': 0.1, 'probability': True, 'kernel': 'poly', 'degree': 2, 'coef0': 1},
    'X_num': X_all
}
result = sgml.cv(df_train, skf, hparams, config, svc_adapter)
np.mean(result['valid_scores']), np.mean(result['train_scores'])

Fold:   0%|          | 0/5 [00:00<?, ?it/s]

(0.8931032547699214, 0.9092636433782267)

In [174]:
sfs = sc.cache_result(
    'ff_sfsf_p2svc',
    lambda : mfs.SequentialFeatureSelector(
        estimator = SVC(**hparams['model_params']), k_features = 'best', forward = True, floating = True, scoring = 'roc_auc', cv = skf, n_jobs = -1
    ).fit(df_train[X_all], df_train[target])
)
np.array(sfs.k_feature_names_)

array(['mintemp', 'dewpoint', 'sunshine', 'windspeed', 'chp', 'cloud_1',
       'cloud_3', 'cloud_6', 'dewpoint_2', 'cos_wd_1', 'cos_wd_3',
       'cos_wd_4', 'dewpoint_ma24', 'sunshine_ma24', 'cos_wd_ma24',
       'sin_wd_ma24', 'year'], dtype='<U13')

In [175]:
hparams = {
    'model_params': {'C': 0.1, 'probability': True, 'kernel': 'poly', 'degree': 2, 'coef0': 1},
    'X_num': list(sfs.k_feature_names_)
}
result = sgml.cv(df_train, skf, hparams, config, svc_adapter)
sc.cv_result('p2svc_sfs_f', hparams, svc_adapter, result['valid_prd'])
np.mean(result['valid_scores']), np.mean(result['train_scores'])

Fold:   0%|          | 0/5 [00:00<?, ?it/s]

(0.9009034792368127, 0.9073711069023569)

In [176]:
sfs = sc.cache_result(
    'bf_sfsf_p2svc',
    lambda : mfs.SequentialFeatureSelector(
        estimator = SVC(**hparams['model_params']), k_features = 'best', forward = True, floating = True, scoring = 'roc_auc', cv = skf, n_jobs = -1
    ).fit(df_train[X_all], df_train[target])
)
np.array(sfs.k_feature_names_)


STOPPING EARLY DUE TO KEYBOARD INTERRUPT...

array(None, dtype=object)

In [177]:
hparams = {
    'model_params': {'C': 0.1, 'probability': True, 'kernel': 'poly', 'degree': 2, 'coef0': 1},
    'X_num': list(sfs.k_feature_names_)
}
result = sgml.cv(df_train, skf, hparams, config, svc_adapter)
sc.cv_result('p2svc_bfs_f', hparams, svc_adapter, result['valid_prd'])
np.mean(result['valid_scores']), np.mean(result['train_scores'])

TypeError: 'NoneType' object is not iterable

# Feature Selection Analysis

In [153]:
result_sk = ['ff_sfs_lr', 'ff_sfs_lr2', 'bf_sfs_lr', 'bf_sfs_lr2', 'ff_sfs_lgb', 'ff_sfs_lgb2', 'bf_sfs_lgb', 'ff_sfs_xgb', 'ff_sfs_xgb2', 'bf_sfs_xgb']
selected_sk = [
    np.array(X_all)[sc.read_result(i).get_support()].tolist()
    for i in result_sk
]

In [161]:
pd.Series(selected_sk, index = result_sk).explode().value_counts().iloc[:10]

temparature_ma24    10
sunshine            10
windspeed           10
chp                 10
chp_3                9
year                 9
mintemp              9
dewpoint             9
sunshine_1           8
sin_ed               8
Name: count, dtype: int64

In [168]:
result_mfs = ['ff_sfsf_lr', 'ff_sfsf_lr2', 'bf_sfsf_lr', 'ff_sfs_lgb_f', 'bf_sfs_lgb_f', 'ff_sfs_xgb_f', 'bf_sfs_xgb_f', 'ff_sfsf_lsvc', 'bf_sfsf_lsvc']
selected_mfs = [
    list(sc.read_result(i).k_feature_names_)
    for i in result_mfs
]

In [169]:
pd.Series(selected_mfs, index = result_mfs).explode().value_counts().iloc[:10]

sin_ed              9
mintemp             9
sunshine            9
dewpoint            8
windspeed           8
temparature_ma24    8
chp                 7
humidity_ma24       7
maxtemp_ma24        6
cloud_3             6
Name: count, dtype: int64